In [1]:
import qcodes as qc
station = qc.Station(config_file='config/station.yaml')

In [2]:
dac1 = station.load_mock_dac1()
dac2 = station.load_mock_dac2()

In [3]:
%load_ext instrmcp.extensions

✅ QCoDeS MCP extension loaded
🛡️  Default mode: safe
📋 Use %mcp_status to check server status
⚠️  Use %mcp_unsafe to switch to unsafe mode (if needed)
🚀 Use %mcp_start to start the server


In [4]:
%mcp_status

🛡️ MCP Server Status:
   Desired Mode: safe
   Server Instance: ❌ Not created yet
   Available: %mcp_start


In [5]:
%mcp_option add measureit database auto_correct_json

ℹ️  Already enabled: measureit
ℹ️  Already enabled: database
ℹ️  Already enabled: auto_correct_json
ℹ️  No option changes applied.


In [6]:
%mcp_unsafe

⚠️  Mode set to unsafe
⚠️  UNSAFE MODE: execute_editing_cell tool will be available
✅ Mode will take effect when server starts


In [7]:
%mcp_dangerous

⚠️⚠️⚠️  DANGEROUS MODE ENABLED  ⚠️⚠️⚠️
All consent dialogs will be automatically approved!
This mode bypasses all safety confirmations.
✅ Mode will take effect when server starts


In [8]:
%mcp_start

/Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.11/site-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/Users/caijiaqi/miniforge3/envs/instrMCPdev/lib/python3.11/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [18120]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8123 (Press CTRL+C to quit)


🚀 Starting MCP server...
✅ MCP server started in ☠️ dangerous mode
   Running on http://127.0.0.1:8123
⚠️⚠️⚠️  All consent dialogs auto-approved!
INFO:     127.0.0.1:63610 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63610 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:63611 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63611 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:63611 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63611 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63612 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63613 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:63614 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63615 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63616 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63617 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63618 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:63619 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:63620 - "POST /mcp HTTP/1.

print("Hello, testing output!")

In [18]:
# Create a 120s 0D sweep following dac1.ch01.voltage
from measureit import Sweep0D

sweep_0d = Sweep0D(3, inter_delay=0.1, save_data=False, plot_data=True)
sweep_0d.follow_param(dac1.ch01.voltage)

# Start the sweep (non-blocking)
%gui qt
sweep_0d.start()

In [9]:
import numpy as np
import pandas as pd

x = 42
my_list = [1, 2, 3]
my_dict = {"a": 1, "b": 2}
class MyClass: pass
obj = MyClass()

arr = np.array([[1, 2, 3], [4, 5, 6]])
df = pd.DataFrame({'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']})
large_arr = np.zeros((1000, 1000))

In [10]:
print("aaa world")
print("hello world2")
print("hello world3")

hello world


x=1

In [ ]:
my_list

In [ ]:
# new cell